## Import Libraries


In [17]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt 
%matplotlib
import seaborn as sns 

Using matplotlib backend: TkAgg


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from glob import glob


In [4]:
IMAGE_SIZE = [224,224]
train_path = 'Dataset/train'
test_path = 'Dataset/test'

In [5]:
resent = ResNet50(input_shape=IMAGE_SIZE +[3],weights='imagenet',include_top= False)

In [6]:
# I need to train the last layer
for layer in resent.layers:
    layer.trainable = False

In [7]:
#number of output classes
folders =glob('Dataset/train/*')

In [8]:
x = Flatten()(resent.output)

In [9]:
# output layer
prediction = Dense(len(folders),activation = 'softmax')(x)
# create a model 
model = Model(inputs=resent.input,outputs=prediction)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [11]:
# cost and optimizer methods
model.compile(
    loss='categorical_crossentropy',
    optimizer = 'adam',
    metrics=['accuracy']

)

In [12]:
# Image data generator 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [13]:
train_set  =train_datagen.flow_from_directory('Dataset/train',
                                             target_size=(224,224),
                                             batch_size=32,
                                             class_mode='categorical')

Found 1951 images belonging to 4 classes.


In [14]:
test_set  =test_datagen.flow_from_directory('Dataset/test',
                                             target_size=(224,224),
                                             batch_size=32,
                                             class_mode='categorical')

Found 18 images belonging to 4 classes.


In [15]:
# fit the model
r = model.fit(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)

)

Epoch 1/20
61/61 [==============================] - 109s 2s/step - loss: 3.3866 - accuracy: 0.4136 - val_loss: 1.2434 - val_accuracy: 0.5556
Epoch 2/20
61/61 [==============================] - 104s 2s/step - loss: 1.1445 - accuracy: 0.5372 - val_loss: 0.8802 - val_accuracy: 0.7222
Epoch 3/20
61/61 [==============================] - 106s 2s/step - loss: 0.9249 - accuracy: 0.6192 - val_loss: 0.7918 - val_accuracy: 0.6111
Epoch 4/20
61/61 [==============================] - 105s 2s/step - loss: 0.8071 - accuracy: 0.6776 - val_loss: 0.6613 - val_accuracy: 0.7222
Epoch 5/20
61/61 [==============================] - 106s 2s/step - loss: 0.9278 - accuracy: 0.6284 - val_loss: 0.7437 - val_accuracy: 0.6667
Epoch 6/20
61/61 [==============================] - 107s 2s/step - loss: 1.1167 - accuracy: 0.6043 - val_loss: 1.1412 - val_accuracy: 0.6111
Epoch 7/20
61/61 [==============================] - 106s 2s/step - loss: 1.0780 - accuracy: 0.6197 - val_loss: 0.7218 - val_accuracy: 0.7222
Epoch 8/20
61

In [19]:
# plot the accuracy
plt.plot(r.history['loss'],label ='loss' )
plt.plot(r.history['val_loss'],label ='val_loss' )
plt.legend()
plt.show()
plt.savefig('loss & val_loss')


In [18]:
# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [20]:
y_pred = model.predict(test_set)

In [ ]:
y_pred = np.argmax(y_pred, axis=1)

In [22]:
# save it as a h5 file
from tensorflow.keras.models import load_model
model.save('model_resnet50.h5')